Course: Computational Statistics | SS2020 | University of Bonn
# Topic: Comparison in algorithms for Classification Tree

Author: Po-Tse Cheng

maybe add some structure here? (see if we got enough space)

In [5]:
#install packages in need
install.packages(c("partykit", "RWeka", "rpart", "truncnorm", "remotes","wakefield", "partykit", "data.table"))

#initial the packages require for the projects:
library(wakefield)
library(partykit)
library(truncnorm)
library(rpart)
library(RWeka)
library(partykit)
library(data.table)

#import the custom DGP function from R codes in the repository
source("R codes/DGP.R")

#supress unwanted warning in loading packages
#options(warn=-1)
#do this in the end

Installing packages into 'C:/Users/Viktor Cheng/Documents/R/win-library/3.6'
(as 'lib' is unspecified)



  There is a binary version available but the source version is later:
      binary source needs_compilation
RWeka 0.4-42 0.4-43             FALSE



Warning message:
"packages 'partykit', 'rpart', 'truncnorm', 'wakefield', 'data.table' are in use and will not be installed"

package 'remotes' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Viktor Cheng\AppData\Local\Temp\RtmpuAN6JS\downloaded_packages


installing the source package 'RWeka'

Warning message in install.packages(c("partykit", "RWeka", "rpart", "truncnorm", :
"installation of package 'RWeka' had non-zero exit status"

## 1. Introduction

The main purpose of this notebook is to showcase the strengh and weekness of different algorithms in classification tree, in particular, the decision making mechanism behind the splits between nodes. The motivation behind this attempt is to understand each individual splitting methods more thoroughly, as within the courses


Two seperate DGPs with different characteristics will be utilised to help evaulate the algorithms. In total 3( if got time 4) will be discussed:
<ul>
    <li>Classification and Regression Trees (CART)</li>
    <li>C4.5</li>
    <li>Conditional Inference Trees (Ctree)</li>
</ul>

The most direct algorithm for determining a split is the <em>Gini Index</em>, which is employed by CART. C4.5 uses the <em>Shannon Entropy </em>method which in some sense is simliar to Gini Index. Lastly, Ctree incoporates the <em>Chi-square test</em> to determine the split. In this notebook,  we will be using the rpart package for CART algorithm, rweka for J48, and finally partykit package for Ctree. 

This notebook is also constructed in the following way: in the second chapter we will take a look at the two DGPs including personal assumptions and the mechanism of each algorithms. Section 3 is using the examples 



<div class="alert alert-block alert-info">
    <b>Additional Note:</b> The DGP functions are quite long, therefore I decided to import it from the repository. For the sake of transparency, please <a href="https://github.com/PoTseCheng/Computational-Statistic/blob/master/R%20codes/DGP.R">click here</a> to evaluate the code. 
    
</div>

## 2. DGPs and Theoretical Perspectives

### 2.1. Data Generating Procedures

In this section we will be going through the two main DGPs: The first one came from Tian et al.(2020) from Journal of Infection, while the second one is from Jones-White et al.(2014) published in Research in Higher Education. The first paper analysed the symptoms of COVID-19 and the consequences of all 262 cases. Furthermore, the 262 cases were splitted into 2 subgroups by clinical perspective. The other paper aims to establish a connection between financial aids and students' college outcomes. 


Below is a short summary of data from the original papers:

<table><tr>
<thead>
 <tr><th>Description Statistics 1</th><th>Description Statistics 2</th></tr>   
</thead>
    
<tbody>
    
 <tr><td> <img src="Pics/DGP1.png"> </td><td> <img src="Pics/DGP2.png"> </td></tr>

</tbody>
</table>

These two papers were intentionally selected as they incorporated different types of data. 

For the first paper, no correlations between all variables, rather cases were immidiately splitted into sever

Below are the tables for 

|Variables   |Type       |Explaination                                                                                  |
|------------|-----------|----------------------------------------------------------------------------------------------|
|Severity    |Categorical|This variable determines if a patient is diagnosed as a severe case                           |
|Gender      |Categorical|Male or female                                                                                |
|Age         |Numerical  |Age                                                                                           |
|Address     |Numerical  |Where the patient lives                                                                       |
|Contact     |Categorical|Have the patient been in contact with anyone from Wuhan or any infected cases within 14 days  |
|Temperature |Numerical  |Body temerature measured during diagnosis                                                     |
|Tlabel      |Numerical  |Label correspond to temperature, including: Normal, Fever, and Hyperpyrexia                   |
|Symptoms    |Numerical  |Symptoms including Coughing, Fatigue, Dyspnea, Headache (Without fever)                       |
|Result      |Categorical|Three categories: discharge, further hospitalization, death                                   |





|Variables   |Type       |Explaination                                                                                       |
|------------|-----------|---------------------------------------------------------------------------------------------------|
|Gender      |Categorical|Male or female                                                                                     |
|Race        |Categorical|Which race is the student from, including White, Asian, and other Minorities	                     |
|Generation  |Categorical|Denotes if the student is the first generation in the family to attend university or not           |
|Status	     |Categorical|Whether the student is younger than 19 on the first day of enrolment in the university             |
|Geographic  |Categorical|Details if the students are from specific states                                                   |
|Remedy      |Categorical|Shows if the student takes remedial courses or nnot                                                |
|Choice      |Categorical|If the observed university is the students' first choice of university or not                      |
|Living      |Categorical|Where the student live: Outside Campus or on Campus                                                |
|Community   |Categorical|If the student is participating in community activities                                            |
|Athlete     |Categorical|If the student is considered as an athlete representitive for the university                       |
|Aidtype     |Categorical|Which type of financial aid the student gets, including: None, Loan, Need, and Merit               |
|Aidamount   |Numerical  |The amount (in \$1000 USD) that corresponds to the Aidtype, notice for none it is always 0         |
|Unmet       |Numerical  |The amount (in \$1000 USD) of unmet financial needs that the students get after the Aid            |
|ACT         |Numerical  |The score the student gets for composite American College Test (ACT)                               |
|AP          |Numerical  |Number of Advanced Placement credit                                                                |
|Course      |Numerical  |Course completion rate during the first semester                                                   |
|Ccount      |Numerical  |Amount of C counts with the maxium given 5                                                         |
|Dcount      |Numerical  |Amount of D counts with the maxium given 4                                                         |
|Result      |Categorical|The end result correspond to the paper: Graduated, Transferred then graduated, or Not Graduated    |

Here are the assumptions and made regarding first DGP:

<ol>
<li>I assume each individual symptoms are independent with each other. This may not be true in terms of medical science, however, it lessens the complexity of the model immensly. Body temperature on the other hand are connected with the Tlabel, which </li>
    
<li>The results of the patiences are also <strong>not correlated</strong> with all other symptoms. This is also the main difference of this DGP with the other one. </li>
    
    
</ol>

Below are a small portion of generated data from our DGP. Showcase1 denotes the samples from DGP1 and Showcase2 from DGP2.

In [6]:
set.seed(123)
data1<- Observations(100)
data2<- Observations2(100)

In [7]:
showcase1<- data.table(data1)
showcase1[sample(.N, 6)]

Severity,Gender,Age,Address,Contact,Temperature,Tlabel,Symptoms,Result
0,Female,57,Beijin,Case 14 days,37.35,Normal,"Coughing, Fatigue",Hospitalization
0,Female,62,Wuhan,Unknown,38.58,Fever,No Other Symptoms,Hospitalization
0,Male,45,Beijin,Case 14 days,38.43,Fever,No Other Symptoms,Hospitalization
1,Female,27,Beijin,Case 14 days,36.59,Normal,"Coughing, Fatigue",Hospitalization
0,Female,32,Beijin,Wuhan 14 days,37.37,Normal,No Other Symptoms,Hospitalization
0,Female,23,Other,Wuhan 14 days,37.52,Fever,"Coughing, Fatigue",Hospitalization


In [8]:
showcase2<- data.table(data2)
showcase2[sample(.N, 6)]

Gender,Race,Generation,Status,Geographic,Remedy,Choice,Living,Community,Athlete,Aidtype,Aidamount,Unmet,ACT,AP,Course,Ccount,Dcount,Result
Male,White,Yes,Traditional,Reciprocity state,No,First Choice,Outside Campus,No,No,Loan Aid,0.53,5.12,21.36,13.63,80.40,1.12,0.32,Graduated
Male,White,No,Traditional,Reciprocity state,No,First Choice,On Campus,Yes,No,Need Aid,1.20,4.55,25.36,0.87,97.04,0.49,0.48,Graduated
Female,Minorities,Yes,Traditional,In-state,No,First Choice,On Campus,No,No,Loan Aid,2.39,11.67,31.18,10.15,97.34,1.88,0.31,Transferred Graduated
Female,White,No,Traditional,Reciprocity state,No,Others,Outside Campus,No,No,Loan Aid,0.10,4.06,30.59,5.89,74.35,0.56,0.20,Graduated
Female,White,No,Traditional,In-state,No,First Choice,Outside Campus,No,No,Loan Aid,4.35,0.31,19.70,6.28,78.84,2.41,0.14,Graduated
Male,White,No,Traditional,Reciprocity state,No,Others,On Campus,No,No,Merit Aid,0.40,0.50,28.12,13.21,97.38,0.47,0.36,Graduated


### 2.2. Mechanisms and Expectations

We expect that

## Algorithm Evaluation

In this cha

In [ ]:
algorithm: CART

In [ ]:
SOME  OTHER SHIT REGARDING C4.5

In [ ]:
algorithm:c4.5 with J48

In [ ]:
CTree

Benchmark accuracy/ also give ratings

additional chapter (Cruise):
addition since the package is not avaliable in r, one must download the algorithm manually from the ... website.

Reference:

Jones-White, D. R., Radcliffe, P. M., Lorenz, L. M., & Soria, K. M. (2014). Priced out?. Research in Higher Education, 55(4), 329-350.

Loh, W. Y. (2014). Classification and regression tree methods. Wiley StatsRef: Statistics Reference Online.

Loh, W. Y., & Shih, Y. S. (1997). Split selection methods for classification trees. Statistica sinica, 815-840.

Tian, S., Hu, N., Lou, J., Chen, K., Kang, X., Xiang, Z., ... & Chen, G. (2020). Characteristics of COVID-19 infection in Beijing. Journal of Infection.